In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from ml_metrics import rmsle
from sklearn.cross_validation import LabelKFold

In [2]:
train = pd.read_csv('train.csv', parse_dates=['datetime'])
test = pd.read_csv('test.csv', parse_dates=['datetime'])

### Merge train & test
Merge train & test into one dataframe. This allow in more flexible and simiple way preprocessing data for both dataset.

In [3]:
df_all = train.append(test)

df_all['count_log'] = np.log( df_all['count'] + 1 )
df_all['casual_log'] = np.log( df_all['casual'] + 1 )
df_all['registered_log'] = np.log( df_all['registered'] + 1 )

df_all['is_test'] = df_all['count'].isnull()

In [4]:
def select_features(data):
    black_list = ['casual', 'registered', 'count', 'is_test', 'datetime', 'count_log', 'casual_log', 'registered_log']
    return [feat for feat in data.columns if feat not in black_list]
    
def get_X_y(data, target_var='count'):
    features = select_features(data)
    return data[features].values, data[target_var].values

In [6]:
def cat_hour(hour):
    if 5 >= hour < 10:
        return 1#morning
    elif 10 >= hour < 17:
        return 2#day
    elif 17 >= hour < 23:
        return 3 #evening
    else:
        return 4 #night
    
def feature_engineering(data):
    data['year'] = data['datetime'].dt.year
    data['diff_year'] = data['year'] - 2010
    data['month'] = data['datetime'].dt.month
    data['day'] = data['datetime'].dt.day
    data['hour'] = data['datetime'].dt.hour
    data['minute'] = data['datetime'].dt.minute
    data['dayofweek'] = data['datetime'].dt.dayofweek
    data['weekofyear'] = data['datetime'].dt.weekofyear
    data['weekend'] = data.dayofweek.map(lambda x: int(x in [5,6]) )
    data['time_of_day'] = data['hour'].map(cat_hour)
    
    data['dayofyear'] = data['datetime'].dt.dayofyear
    data['day_'] = data[ ['year', 'dayofyear'] ].apply(lambda x: x['dayofyear'] + int(str(x['year'])[-1]) * 365  , axis=1)
    
    data['rush_hour'] = data['datetime'].apply(lambda i: min([np.fabs(9-i.hour), np.fabs(20-i.hour)]))
    data.loc[:,('rush_hour')] = data['datetime'].apply(lambda i: np.fabs(14-i.hour))
    data.loc[data['workingday'] != 0].loc[:,('rush_hour')] = 0
    
    data['holiday'] = data[['month', 'day', 'holiday', 'year']].apply(lambda x: (x['holiday'], 1)[x['year'] == 2012 and x['month'] == 10 and (x['day'] in [30])], axis = 1)
    data['holiday'] = data[['month', 'day', 'holiday']].apply(lambda x: (x['holiday'], 1)[x['month'] == 12 and (x['day'] in [24, 26, 31])], axis = 1)
    
    data['workingday'] = data[['month', 'day', 'workingday']].apply(lambda x: (x['workingday'], 0)[x['month'] == 12 and x['day'] in [24, 31]], axis = 1)
    data['peak'] = data[['hour', 'workingday']].apply(lambda x: (0, 1)[(x['workingday'] == 1 and  ( x['hour'] == 8 or 17 <= x['hour'] <= 18 or 12 <= x['hour'] <= 12)) or (x['workingday'] == 0 and  10 <= x['hour'] <= 19)], axis = 1)
    data['sticky'] = data[['humidity', 'workingday']].apply(lambda x: (0, 1)[x['workingday'] == 1 and x['humidity'] >= 60], axis = 1)

    return data

df_all = feature_engineering(df_all)

## Modeling
Tuning & Ensembling

In [7]:
def modeling(models, data, n_folds=3): #there're 3 folds
    labels = data['month'].values
    
    scores = []
    for n_fold, (train_idx, test_idx) in enumerate(LabelKFold(labels, n_folds=n_folds)):
        print("n_fold: ", n_fold)
        y_pred = np.array([0.] * len(test_idx))
        
        for weight_for_model, feats, model in models:
            X = data[feats].values
            
            X_train, X_test = X[train_idx], X[test_idx]
            y_reg_train = data['registered_log'][train_idx].astype('float')
            y_cas_train = data['casual_log'][train_idx].astype('float')
            y_test = data['count'][test_idx].astype('float')

            
        
            model.fit(X_train, y_reg_train)
            y_pred_reg_log = model.predict(X_test)
            y_pred_reg = np.exp( y_pred_reg_log ) - 1

            model.fit(X_train, y_cas_train)
            y_pred_cas_log = model.predict(X_test)
            y_pred_cas = np.exp( y_pred_cas_log ) - 1

            y_pred +=  weight_for_model * (y_pred_reg + y_pred_cas)
            
        score = rmsle(y_test, y_pred)
        scores.append(score)
        
    print("score: {0}, std-score: {1}".format( round(np.mean(scores), 5), round(np.std(scores), 4) ))
        

xgb_params = {'n_estimators':150,  'learning_rate':0.1, 'max_depth':5, 'subsample':0.6, 'colsample_bytree': 0.8}
xgb_model = xgb.XGBRegressor(**xgb_params)

gbm_params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**gbm_params)

rf_params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**rf_params)

xgb_feats = ['weather', 'temp', 'atemp', 'humidity', 'windspeed', 'holiday', 'workingday', 'season','hour', 'dayofweek', 'year']
gbm_feats = ['weather', 'temp', 'atemp', 'humidity', 'windspeed', 'holiday', 'workingday', 'season','hour', 'dayofweek', 'year']
rf_feats = ['weather', 'temp', 'atemp', 'windspeed','workingday', 'season', 'holiday', 'hour', 'dayofweek', 'weekofyear', 'rush_hour', 'peak']

models = [
    #weight, features, model
    #weight in sumary should be equal 1
    
    (.8 * .7, xgb_feats, xgb_model),
    (.8 * .3, gbm_feats, gbm_model),
    (.2, rf_feats, rf_model),
]

modeling(models, df_all[~df_all.is_test])

('n_fold: ', 0)
('n_fold: ', 1)
('n_fold: ', 2)
score: 0.33141, std-score: 0.01


##  Submit

In [9]:
feats = ['weather', 'temp', 'atemp', 'humidity', 'windspeed', 'holiday', 'workingday', 'season','hour', 'dayofweek', 'year']
xgb_features = feats
gbm_features = feats
rf_features = ['weather', 'temp', 'atemp', 'windspeed','workingday', 'season', 'holiday', 'hour', 'dayofweek', 'weekofyear', 'rush_hour', 'peak']
et_features = rf_features

df_train = df_all[~df_all['count'].isnull()]
df_test =  df_all[ df_all['count'].isnull()]                 

_, y_train_count = get_X_y(df_train, 'count_log')
(_, y_train_reg), (_, y_train_cas) = get_X_y(df_train, 'registered_log'), get_X_y(df_train, 'casual_log')

gbm_X_train, xgb_X_train = df_train[gbm_features], df_train[xgb_features]
rf_X_train, et_X_train = df_train[rf_features], df_train[rf_features]

gbm_X_test, xgb_X_test  = df_test[gbm_features], df_test[xgb_features]
rf_X_test, et_X_test  = df_test[rf_features], df_test[rf_features]

gbm_params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
rf_params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
xgb_params = {'n_estimators':150,  'learning_rate':0.1, 'max_depth':5, 'subsample':0.6, 'colsample_bytree': 0.8}
#et_params = {'n_estimators': 100, 'min_samples_leaf': 5, 'random_state': 0, 'min_samples_split': 2}


def predict(X_train, X_test, model):
    model.fit(X_train, y_train_count)
    y_pred_count_log = model.predict(X_test)
    y_pred_count = np.exp( y_pred_count_log ) - 1
    
    model.fit(X_train, y_train_reg)
    y_pred_reg_log = model.predict(X_test)
    y_pred_reg = np.exp( y_pred_reg_log ) - 1

    model.fit(X_train, y_train_cas)
    y_pred_cas_log = model.predict(X_test)
    y_pred_cas = np.exp( y_pred_cas_log ) - 1
    
    return y_pred_reg + y_pred_cas
    #return .3*y_pred_count + .7*(y_pred_reg + y_pred_cas)

gbm_count = predict(gbm_X_train, gbm_X_test, GradientBoostingRegressor(**gbm_params))
rf_count = predict(rf_X_train, rf_X_test, RandomForestRegressor(**rf_params))
xgb_count = predict(xgb_X_train, xgb_X_test, xgb.XGBRegressor(**xgb_params))
#et_count = predict(et_X_train, et_X_test, ExtraTreesRegressor(**et_params))

test['count'] = .2*rf_count + .8*( .3*gbm_count  + .7*xgb_count)

In [10]:
test[ ['datetime', 'count'] ].to_csv('final_submit.csv', index=False)